In [ ]:
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
from datetime import date
from selenium.webdriver.common.by import By
import re

#Make sure you have chrome webdriver installed
#If not, go to your chrome browser and type about::
#That would give you  the version number then google "chrome web driver version x" and download it
#Store it in a folder called "Driver". The "Driver" folder must be in the same folder as this file.

#This is searching for the webdriver in your folder
browser = webdriver.Chrome('Driver/chromedriver.exe')
browser.get('https://www.linkedin.com/uas/login')
username = '' #Provide your own
password = '' #Provide your own
elementID = browser.find_element(By.ID,'username')
elementID.send_keys(username)
elementID = browser.find_element(By.ID,'password')
elementID.send_keys(password)
elementID.submit()

In [ ]:
url = [''] #List of URL's, provide your own. Must be strings

In [ ]:
def profile_scrap2(url):
    #First we scrap all the details on the main page
    #Then we check the attributes for whom the external buttons exist
    #The attributes without external buttones, those elements are extracted
    #Then we visit the external sites

    browser.get(url)
    src = browser.page_source
    soup = bs(src)

    name = []
    location = []
    header = []
    connection = []
    about =[]
    today = [date.today()]
    exp_button = False

    #STEP 1 : Acquire all data on main page
    #ABOUT TAG

    #The ember 202 thing was geting different results for different profiles, I simply replaced it to the below div tag to make it better
    if bool(soup.find('div', class_ = "pv-shared-text-with-see-more t-14 t-normal t-black display-flex align-items-center")) == True:
        about_tag = soup.find('div', class_ = "pv-shared-text-with-see-more t-14 t-normal t-black display-flex align-items-center")
    else:
        about_tag = ''

    if about_tag == "":
        about.append("Not given")
    else:
        about.append(about_tag.text.strip())

    #NAME TAG

    name_tag = soup.find('h1', class_ = 'text-heading-xlarge inline t-24 v-align-middle break-words')
    name.append(name_tag.text)

    #LOCATION TAG
    location_tag = soup.find('span', class_ = 'text-body-small inline t-black--light break-words')
    location.append(location_tag.text.rstrip().strip())

    #HEADER TAG
    header_tag = soup.find('div', class_ = 'text-body-medium break-words')
    if bool(header_tag) == True:
        header.append(header_tag.text.lstrip().rstrip())
    else:
        pass

    
    #STEP - 2 : Check if external buttons exist

    x = soup.findAll('section', class_ = "artdeco-card ember-view break-words pb3 mt4")
    exp_tag = ""
    ed_tag = ""
    cert_tag = ""
    #First we extract the respective div tags which contain information about the needed attributes
    for i in x:
        if i.div.attrs['id']=='education':
            ed_tag = i.find('div', class_ = "pvs-list__outer-container")

        if i.div.attrs['id']=="experience":
            exp_tag =  i.find('div', class_ = "pvs-list__outer-container") 

        if i.div.attrs['id']=="licenses_and_certifications":
            cert_tag = i.find('div', class_ = "pvs-list__outer-container")

    #Checking the buttons

    #Based on previously obtained tags, we check if those buttons exist or not
    #Experience tag
    try: 
        if re.findall(r'See all .+',exp_tag.find('span', class_ = "pvs-navigation__text").text.strip()):
            exp_button = True
    except:
        exp_button = False

    #Education tag
    try: 
        if re.findall(r'See all .+',ed_tag.find('span', class_ = "pvs-navigation__text").text.strip()):
            ed_button = True
    except:
        ed_button = False

    #License tag
    try: 
        if re.findall(r'See all .+',cert_tag.find('span', class_ = "pvs-navigation__text").text.strip()):
            cert_button = True
    except:
        cert_button = False
    




    #STEP - 3 : All the buttons whose value is false, we extract their information

    #Education  Tag
    ed_name = []
    ed_course = []
    ed_year = []
    #ed_activities=[]
    ed_name_string = "" 
    ed_course_string = "" 
    ed_year_string = ""
    if ed_button == False:
        if ed_tag =="":
            ed_name = []
            ed_course = []
            ed_year = []
            #ed_activities=[]
        else:
            for i in ed_tag.findAll('li', class_ = "artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column"):
                #Name of institute

                ed_name_string += i.find('span', class_ = "visually-hidden").text.strip()
                ed_name_string += " | "

                
                #Name of Course
                if bool(i.find('span', class_ = "t-14 t-normal")) == True:
                    ed_course_string += i.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip()
                    ed_course_string += " | "
                else:
                    ed_course.append("Not Given")

                #Year of course
                if bool(i.find('span', class_ = "t-14 t-normal t-black--light")) == True:
                    ed_year_string += i.find('span', class_ = "t-14 t-normal t-black--light").find('span', class_ = "visually-hidden").text.strip()
                    ed_year_string += " | "
                else:
                    ed_year.append("Not Given")
                
    if bool(ed_year_string) == True:
        ed_year.append(ed_year_string)

    if bool(ed_name_string) == True:
        ed_name.append(ed_name_string)

    if bool(ed_course_string) == True:
        ed_course.append(ed_course_string)

    preCompCompiled = ""
    #Experience Tag
    curr_company = []
    curr_yearLocation = []
    curr_position = []
    pre_comp = []
    pre_yearLocation = []
    
    if exp_button == False:
        if exp_tag == "":
            curr_company = []
            curr_yearLocation = []
            curr_position = []
            pre_comp = []
            pre_yearLocation = []

        else:
            expTag = exp_tag.find('ul',class_ = "pvs-list ph5 display-flex flex-row flex-wrap").findAll('li', class_ = "artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column")

            curr = expTag[0]

            #flag tells us if our company has multiple positions or single
            flag = bool(curr.find('a', class_ = "optional-action-target-wrapper display-flex flex-column full-width"))

            #if multiple positions exist
            if flag:
                curr_company.append(curr.find('span', class_ = "visually-hidden").text.strip())
                curr_yearLocation.append(curr.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip())
                curr_position.append(curr.find('li').find('span', class_ = "visually-hidden").text.strip())
            else:
                curr_company.append(curr.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip())
                curr_yearLocation.append(curr.find('span', class_ = "t-14 t-normal t-black--light").find('span', class_  ="visually-hidden").text.strip())
                curr_position.append(curr.find('span', class_ = "t-bold mr1").find('span', class_ = "visually-hidden").text.strip())



            #Previous companies
            for i in expTag[1:]:
                flag = bool(i.find('a', class_ = "optional-action-target-wrapper display-flex flex-column full-width"))

                if flag:
                    pre_comp.append(i.find('span', class_ = "visually-hidden").text.strip())
                    pre_yearLocation.append(i.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip())
                else:
                    pre_comp.append(i.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip())
                    pre_yearLocation.append(i.find('span', class_ = "t-14 t-normal t-black--light").find('span', class_  ="visually-hidden").text.strip())

            preCompCompiled = ""
            for i in pre_comp:
                preCompCompiled+=i
                preCompCompiled+=" |"
            preCompCompiled = preCompCompiled[:len(preCompCompiled)-1]
            preYearCompiled = ""
            for i in pre_yearLocation:
                preYearCompiled+=i
                preYearCompiled+=" |"
            preYearCompiled = preYearCompiled[:len(preYearCompiled)-1]

                
            

    #Now we move on to acquiring the info that is external


    #STEP - 4 : Acquiring external data

    #Experience tag
    
    if exp_button == True:
        urlExp = url + "details/experience/"
        browser.get(urlExp)
        src = browser.page_source
        soup = bs(src)

        expTag = soup.find('section', class_ ="artdeco-card ember-view pb3").findAll('li', class_ = "pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated")

        curr = expTag[0]

        #flag tells us if our company has multiple positions or single
        flag = bool(curr.find('a', class_ = "optional-action-target-wrapper display-flex flex-column full-width"))

        #if multiple positions exist
        if flag:
            curr_company.append(curr.find('span', class_ = "visually-hidden").text.strip())
            curr_yearLocation.append(curr.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip())
            curr_position.append(curr.find('li').find('span', class_ = "visually-hidden").text.strip())
        else:
            curr_company.append(curr.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip())
            curr_yearLocation.append(curr.find('span', class_ = "t-14 t-normal t-black--light").find('span', class_  ="visually-hidden").text.strip())
            curr_position.append(curr.find('span', class_ = "t-bold mr1").find('span', class_ = "visually-hidden").text.strip())



        #Previous companies
        for i in expTag[1:]:
            flag = bool(i.find('a', class_ = "optional-action-target-wrapper display-flex flex-column full-width"))

            if flag:
                pre_comp.append(i.find('span', class_ = "visually-hidden").text.strip())
                pre_yearLocation.append(i.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip())
            else:
                pre_comp.append(i.find('span', class_ = "t-14 t-normal").find('span', class_ = "visually-hidden").text.strip())
                pre_yearLocation.append(i.find('span', class_ = "t-14 t-normal t-black--light").find('span', class_  ="visually-hidden").text.strip())

        preCompCompiled = ""
        for i in pre_comp:
            preCompCompiled+=i
            preCompCompiled+=" |"
        preCompCompiled = preCompCompiled[:len(preCompCompiled)-1]
        preYearCompiled = ""
        for i in pre_yearLocation:
            preYearCompiled+=i
            preYearCompiled+=" |"
        preYearCompiled = preYearCompiled[:len(preYearCompiled)-1]


        
    final_dict = {"Date of extraction":today,
                "Name": name, 
                "Location" : location,
                "Header" : header,
                "About" : about, 
                "Companies Worked at" : [preCompCompiled],
                "Previous Companies Duration" : [preYearCompiled],
                #"Duration":compYearLocation[1:], #1: because we need the current company at the top
                #"Companies About" : compDescription[1:],
                "Current Company":curr_company,
                "Current Position": curr_position,
                #"Current Company About":[compDescription[0]],
                "Current Duration" : curr_yearLocation,
                #"Current Details": [str(str(time_list[0]) + "," + str(exp_years[0]))],
                #"Previous Companies": comp[1:],
                #"Duration":time_list[1:],
                #"Experience in years" : exp_years[1:],
                "Educational Institutions": ed_name,
                "Course":ed_course,
                #"Stream":ed_stream,
                "Educational Duration":ed_year,
                "Linkedin URL": url}
                #"Grades":ed_grade,
                #"Activity":ed_activities}
                #"Projects": projName,
                #"Dates":projTime,
                #"SKills":[skillCell]}
    return(final_dict)

#This function actually scraps every piece of information present in the linkedin profile, I commented the ones I dont use. You can uncomment them and also uncomment them from the final dictionary

def dict_df(url):
    x = []
    for i in url:
        x.append(pd.DataFrame(dict([(k,pd.Series(v,dtype='object')) for k,v in profile_scrap2(i).items()])))
    df = x[0]
    if len(url)>1:    
        for i in range(1,len(url)):
            df = df.append(x[i])
    else:
        pass
    return df

In [ ]:
df = dict_df(url) #Note, this is a list of URLS as we defined in the second block
df.to_csv('Scraper.csv', encoding = 'utf-8')